In [1]:
# %matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import glob
import itertools
import os

from run import Run
from runs_db import db as runs_db

[('martak', 108), ('gcp', 62), ('deepnet2', 83), ('fiit-gcp-1', 10), ('fiit-gcp-2', 13), ('fiit-gcp-3', 14), ('acer', 32), ('deepnet2070', 10)]


In [2]:
log_path = '/home/fiit/logs'
runs = []

for server in runs_db:
    paths = glob.glob(os.path.join(log_path, server, '*'))
    paths = iter(sorted(paths))

    try:
        for (number, type_, code) in runs_db[server]:
            for _ in range(number):
                try:
                    path = next(paths)
                    runs.append(Run(path, type_, code))
                except KeyError:
                    print(path)
    except StopIteration:
        pass


/home/fiit/logs/deepnet2070/2019-03-29-050040


In [3]:
tasks = ['dep', 'lmo', 'ner', 'pos']
langs = ['cs', 'de', 'en', 'es']

task_metr = {
    'dep': 'las',
    'lmo': 'perplexity',
    'ner': 'chunk_f1',
    'pos': 'acc'
}

task_max = {
    'dep': True,
    'lmo': False,
    'ner': True,
    'pos': True
}

def draw_graphs(metric_func, tasks, langs, role, run_codes=None, run_types=None):
    fig, axes = plt.subplots(len(tasks), len(langs), figsize=(5*len(langs), 4*len(tasks)), squeeze=False)

    relevant_runs = ['vanilla']

    for task, lang in itertools.product(tasks, langs):
        for run in runs:
            if (
                run.contains(task, lang) and
                (run_codes is None or run.code in run_codes) and
                (run_types is None or run.type in run_types)
            ):

                history = run.history(
                    metric=metric_func(task),
                    task=task,
                    language=lang,
                    role=role)
                axes[tasks.index(task), langs.index(lang)].plot(list(history), label=f'{run.code}-{run.type}')

    for ax, col in zip(axes[0], langs):
        ax.set_title(col)

    for ax, row in zip(axes[:, 0], tasks):
        ax.set_ylabel(row, rotation=0, size='large')
        
    for ax_row in axes:
        for ax in ax_row:
            ax.legend()

    plt.show()
    
def print_results(metric_func, metric_max_func, tasks, langs, run_codes=None, run_types=None, print_lambda=None):
    
    if print_lambda is None:
        print_lambda = lambda val, run: print(f'{val:.2f}')
        
    output = []
    
    for task, lang in itertools.product(tasks, langs):
        for run in runs:
            if (
                run.contains(task, lang) and
                (run_codes is None or run.code in run_codes) and
                (run_types is None or run.type in run_types)
            ):
                result = run.metric_eval(
                    metric=metric_func(task),
                    max_=metric_max_func(task),
                    task=task,
                    language=lang)
                res = result[0]
                if res < 1.01:
                    res *= 100
                print_lambda(res, run)
                output.append((res, run))
                
    return output

In [12]:

pair_runs = [run for run in runs if run.code == 'one-aux']
for task, lang in itertools.product(['dep', 'ner', 'pos'], ['cs', 'es']):
    results = []
    for r in pair_runs:
        if r.contains(task, lang):
            score, _ = r.metric_eval(task_metr[task], task_max[task], task=task, language=lang)
            l = list(r.hparams['tasks'])
            l.remove((task, lang))
            results.append((score, l[0]))
    print(task, lang)
    print(sorted(results, key=lambda x: -x[0])[:3])

dep cs
[(0.8879265897728523, ('dep', 'es')), (0.8871039688562673, ('dep', 'es')), (0.8863045203598677, ('dep', 'de'))]
dep es
[(0.8670044331855605, ('dep', 'en')), (0.8667330136614494, ('dep', 'cs')), (0.865647335565005, ('dep', 'de'))]
ner cs
[(0.7877138413685848, ('dep', 'cs')), (0.7861635220125787, ('ner', 'en')), (0.7839433293978749, ('pos', 'cs'))]
ner es
[(0.8768634466308887, ('lmo', 'es')), (0.872039326679577, ('ner', 'de')), (0.8662192393736018, ('ner', 'cs'))]
pos cs
[(99.0360273202834, ('dep', 'cs')), (98.97635833830574, ('lmo', 'cs')), (98.9694066122501, ('ner', 'cs'))]
pos es
[(96.8605808377816, ('pos', 'en')), (96.7701076630779, ('pos', 'de')), (96.75201302813716, ('pos', 'cs'))]


In [ ]:
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['pos'], ['sk'],
    print_lambda=lambda val, run: print(f'{val:.2f} {run.path}'))
print()
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep'], ['sk'],
    print_lambda=lambda val, run: print(f'{val:.2f} {run.path}'))

In [ ]:
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep'], ['cs'],
     ['private-focused-0.75'], ['ml'], lambda val, run: print(f'{val:.2f} {run.path}'))

print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep'], ['es'],
     ['private-focused'], ['ml'], lambda val, run: print(f'{val:.2f} {run.path}'))

print_results(lambda task: task_metr[task], lambda task: task_max[task], ['ner'], ['cs'],
     ['vanilla'], ['mt'], lambda val, run: print(f'{val:.2f} {run.path}'))

print('87.69 /home/fiit/logs/acer/2019-02-22-025619')

print_results(lambda task: task_metr[task], lambda task: task_max[task], ['pos'], ['cs'],
     ['no-adv-tsh-focused-0.75'], ['mt'], lambda val, run: print(f'{val:.2f} {run.path}'))

print_results(lambda task: task_metr[task], lambda task: task_max[task], ['pos'], ['es'],
     ['no-adv-tsh-focused'], ['mt'], lambda val, run: print(f'{val:.2f} {run.path}'))

print()

print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep'], ['de'],
     ['private'], ['mt'], lambda val, run: print(f'{val:.2f} {run.path}'))

print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep'], ['en'],
     ['private'], ['ml'], lambda val, run: print(f'{val:.2f} {run.path}'))

print('83.07 /home/fiit/logs/deepnet2/2019-02-26-041529')
print('85.07 /home/fiit/logs/acer/2019-02-20-213852')

print_results(lambda task: task_metr[task], lambda task: task_max[task], ['pos'], ['de'],
     ['vanilla'], ['mt'], print_lambda=lambda val, run: print(f'{val:.2f} {run.path}'))

print_results(lambda task: task_metr[task], lambda task: task_max[task], ['pos'], ['en'],
     ['private'], ['mt'], print_lambda=lambda val, run: print(f'{val:.2f} {run.path}'))

In [ ]:
# draw_graphs(lambda task: task_metr[task], tasks, langs, 'test',
#              ['private-focused-200', 'no-adv-tsh-200'], ['ml'])
draw_graphs(lambda task: task_metr[task], tasks, langs, 'test',
             ['vanilla', 'no-mwe'], ['ml'])

In [ ]:
pairs = [('dep','cs'), ('dep','es'), ('ner','cs'), ('ner','es'), ('pos','cs'), ('pos','es'),]

def print_row(code, type_):
    for t, l in pairs:
        print_results(lambda task: task_metr[task], lambda task: task_max[task], [t], [l],
                    [code], [type_], lambda val, run: print(f'{val:.2f}', end=' '))

codes = ['private-focused', 'no-adv-tsh-focused']
types = ['ml', 'mt']

print('baseline 89.16 86.51 78.10 86.77 98.89 96.46')
print('best unfocused 89.16 86.70 81.58 87.69 99.04 96.87')


for code in codes:
    for type_ in types:
        print(f'{code}{type_}', end='\t')
        print_row(code, type_)
        print()
        
codes = ['private-focused-0.75', 'no-adv-tsh-focused-0.75']
types = ['ml', 'mt']

for code in codes:
    for type_ in types:
        print(f'{code}{type_}', end='\t\t')
        print_row(code, type_)
        print()
        
codes = ['private-focused-0.75', 'no-adv-tsh-focused-0.75', 'private-focused', 'no-adv-tsh-focused']
types = ['mtml']

for code in codes:
    for type_ in types:
        print(f'{code}{type_}', end='\t\t')
        print_row(code, type_)
        print()


# print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep'], ['en'],
#             ['dep-adv-lambda-0.25'])
# print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep'], ['en'],
#             ['dep-adv-lambda-0.125'])
# print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep'], ['en'],
#             ['no-adv-task-sharing'], ['ml'])

In [ ]:
import numpy as np

pairs = list(itertools.product(['dep', 'ner', 'pos'], langs))

def print_row(code, type_, msg):
    print(msg, end=' ')
    for t, l in pairs:
        print_results(lambda task: task_metr[task], lambda task: task_max[task], [t], [l],
                    [code], [type_], lambda val, run: print(f'{val:.2f}', end=' '))
    print()
        
print_row('vanilla', 'stsl', 'Baseline')

print('Best1', end=' ')
for t, l in pairs:
    output = print_results(lambda task: task_metr[task], lambda task: task_max[task], [t], [l],
                ['one-aux'], print_lambda=lambda _, __: ...)
    print(f'{max([r[0] for r in output]):.2f}', end=' ')
print()

print('Avg', end=' ')
for t, l in pairs:
    output = print_results(lambda task: task_metr[task], lambda task: task_max[task], [t], [l],
                ['one-aux'], print_lambda=lambda _, __: ...)
    print(f'{np.mean([r[0] for r in output]):.2f}', end=' ')
print()


print('Worst1', end=' ')
for t, l in pairs:
    output = print_results(lambda task: task_metr[task], lambda task: task_max[task], [t], [l],
                ['one-aux'], print_lambda=lambda _, __: ...)
    print(f'{min([r[0] for r in output]):.2f}', end=' ')
print()

print_row('vanilla', 'mt', 'A-MT')
print_row('no-adv-task-sharing', 'ml', 'A-ML')
print_row('no-adv-task-sharing', 'mtml', 'A-MTML')
print_row('private', 'mt', 'B-MT')
print_row('private', 'ml', 'B-ML')
print_row('private', 'mtml', 'B-MTML')
print_row('no-adv', 'ml', 'C-ML')
print_row('no-adv', 'mtml', 'C-MTML')

In [ ]:
for t, l in pairs:
    output = print_results(lambda task: task_metr[task], lambda task: task_max[task], [t], [l],
                ['one-aux'], print_lambda=lambda _, __: ...)
    output = sorted(output, key=lambda r: -r[0])
    print(output[0][1].hparams['tasks'])
    #print(f'{max([r[0] for r in output]):.2f}', end=' ')
print()

In [ ]:
draw_graphs(lambda task: task_metr[task], tasks, langs, 'test', ['vanilla'])

In [ ]:
print_results(lambda task: task_metr[task], lambda task: task_max[task], tasks, langs,
     ['fine-tune'], print_lambda=lambda val, run: print(f'{val:.2f} {run.hparams["train_only"]}'))

In [ ]:
def yield_results(task, lang):
    
    code_types = [
        ('vanilla-200', 'stsl'),
        ('vanilla-2000', 'stsl'),
        ('vanilla', 'stsl'),
        ('no-adv-tsh-200', 'mt'),
        ('no-adv-tsh-2000', 'mt'),
        ('vanilla', 'mt'),
        ('no-adv-tsh-200', 'ml'),
        ('no-adv-tsh-2000', 'ml'),
        ('no-adv-task-sharing', 'ml'),
        ('no-adv-tsh-200', 'mtml'),
        ('no-adv-tsh-2000', 'mtml'),
        ('no-adv-task-sharing', 'mtml'),

        
    ]
    
    for code, type_ in code_types:

        output = print_results(lambda task: task_metr[task], lambda task: task_max[task], [task], [lang],
         [code], [type_], print_lambda=lambda val, run: ...)
        for res, run in output:
            if run.hparams['limited_task_language'] in [f'{task}-{lang}', None]:
                yield res

fig, axes = plt.subplots(1, 6, figsize=(20, 3), squeeze=False)

sizes = {
    ('dep', 'cs'): 67900,
    ('dep', 'es'): 14000,
    ('ner', 'cs'): 7100,
    ('ner', 'es'): 6900, 
    ('pos', 'cs'): 67900,  
    ('pos', 'es'): 14000,
}

titles = {
    ('dep', 'cs'): 'DP-cs',
    ('dep', 'es'): 'DP-es',
    ('ner', 'cs'): 'NER-cs',
    ('ner', 'es'): 'NER-es', 
    ('pos', 'cs'): 'POS-cs',  
    ('pos', 'es'): 'POS-es',    
}

for i, (t, l) in enumerate(itertools.product(['dep', 'ner', 'pos'], ['cs', 'es'])):
    results = list(yield_results(t, l))
    axes[0, i].plot([200,2000,sizes[t, l]], results[0:3])
    axes[0, i].plot([200,2000,sizes[t, l]], results[3:6])
    axes[0, i].plot([200,2000,sizes[t, l]], results[6:9])  
    axes[0, i].plot([200,2000,sizes[t, l]], results[9:12])
    axes[0, i].set_xscale('log')
    axes[0, i].set_xticks([200,2000,sizes[t, l]])
    axes[0, i].set_xticklabels(['200', '2000', 'full'])
    axes[0, i].set_title(titles[t, l])
    axes[0, i].set_xlabel('train size')

    
axes[0,0].set_ylabel('performance')
    

#           ax.set_title(col)

#     for ax, row in zip(axes[:, 0], tasks):
#         ax.set_ylabel(row, rotation=0, size='large')
        
#     for ax_row in axes:
#         for ax in ax_row:
#             ax.legend()
  

# for ax, col in zip(axes[0], langs):
#     ax.set_title(col)

# for ax, row in zip(axes[:, 0], tasks):
#     ax.set_ylabel(row, rotation=0, size='large')

# for ax_row in axes:
#     for ax in ax_row:
#         ax.legend()

plt.savefig('sample.pdf')
plt.show()



